# iForest

In [2]:
import warnings
warnings.filterwarnings('ignore')

## General libraries

In [3]:
import os
import sys

import pandas as pd
import numpy as np

from os.path import join
import json
import datetime

import shap
from shap_selection import feature_selection

from sklearn.preprocessing import OneHotEncoder
import time

### Load enviroment variables

In [4]:
from dotenv import load_dotenv
load_dotenv('../.env')

code_root = os.environ['CODE_ROOT']
cfg_path = os.environ['CFG_PATH']
data_root = os.environ['DATA_ROOT']

sys.path.insert(0, code_root)

### Specific libraries

In [5]:
from src.load.functions import get_fs_dataset, fs_datasets_hyperparams

from src.model.functions import train_and_evaluate_iforest
#from src.stability.functions import stability_measure_model, stability_measure_shap

## General parameters

In [6]:
n_iter_fs = 1
n_iter = 1
contamination_percentage = [0.8] 
trees = [25, 50, 100]

# Function to calculate median of a list
def calculate_median(numbers_list):
    return np.median(numbers_list)

# Function to calculate mean of a list
def calculate_mean(numbers_list):
    return np.mean(numbers_list)

# Define aggregation criteria for each variable
aggregation_rules = {
    'n_iter': 'max',
    'n_iter_fs': 'max',
    'f1_median': 'mean',
    'recall_median': 'mean',
    'precision_median': 'mean',
    'roc_auc': 'mean',
    'iforest_stab_unif_median': 'median',
    'shap_stab_median': 'median',
    'shap_stab_mean': 'mean',
}

## Arrhythmia

**Dataset source**: http://odds.cs.stonybrook.edu/arrhythmia-dataset/ (data is transformed from .mat to .csv format)

Shebuti Rayana (2016). ODDS Library [http://odds.cs.stonybrook.edu]. Stony Brook, NY: Stony Brook University, Department of Computer Science.

**Additional sources**:

Liu, Fei Tony, Kai Ming Ting, and Zhi-Hua Zhou. “Isolation forest.” 2008 Eighth IEEE International Conference on Data Mining. IEEE, 2008.

K. M. Ting, J. T. S. Chuan, and F. T. Liu. “Mass: A New Ranking Measure for Anomaly Detection.“, IEEE Transactions on Knowledge and Data Engineering, 2009.

F. Keller, E. Muller, K. Bohm.“HiCS: High-contrast subspaces for density-based outlier ranking.” ICDE, 2012.

In [7]:
dataset_id = 'arrhythmia'

In [8]:
data = get_fs_dataset(dataset_id, data_root)

In [9]:
hyper = fs_datasets_hyperparams(dataset_id)

In [10]:
data.shape

(452, 275)

In [11]:
pd.pivot_table(data,
             values = 'Col1',
               index = 'y', 
              aggfunc = 'count')

,Col1
y,
0,386
1,66


In [12]:
excluded_cols = ['Col15', 'Col63', 'Col65', 'Col79', 'Col127', 'Col128','Col135', 'Col137', 'Col139','Col141','Col147', 'Col152', 'Col153', 'Col160', 'Col200', 'Col260', 'Col270']

### iForest

## Parameters

In [13]:
# Example usage:
path_fi_shap = os.path.join(data_root, "outputs", f"{dataset_id}_fi_shap.parquet")
path_shap = os.path.join(data_root, "outputs", f"{dataset_id}_shap.parquet")

## Feature selection by SHAP

In [18]:
fi_shap_all = pd.read_parquet(path_fi_shap)

In [22]:
# Capture the start time
start_time = datetime.datetime.now()

df = train_and_evaluate_iforest(data, dataset_id=dataset_id, hyper=hyper, fi_df=fi_shap_all, n_tree_estimators=trees, contamination_percentage=contamination_percentage, excluded_cols=excluded_cols, n_iter_fs=n_iter_fs, n_iter=n_iter)

# Capture the finish time
finish_time = datetime.datetime.now()

# Calculate the duration
duration = finish_time - start_time

print(f"Duration: {duration}")

Iteration by tree number: 25
  Iteration by contamination: 0.094
    Number of featured: 6
    Number of featured: 15
    Number of featured: 25
    Number of featured: 38
    Number of featured: 54
    Number of featured: 72
    Number of featured: 93
    Number of featured: 119
    Number of featured: 153
    Number of featured: 233
    Number of featured: 257
Iteration by tree number: 50
  Iteration by contamination: 0.094
    Number of featured: 6
    Number of featured: 15
    Number of featured: 25
    Number of featured: 38
    Number of featured: 54
    Number of featured: 72
    Number of featured: 93
    Number of featured: 119
    Number of featured: 153
    Number of featured: 233
    Number of featured: 257
Iteration by tree number: 100
  Iteration by contamination: 0.094
    Number of featured: 6
    Number of featured: 15
    Number of featured: 25
    Number of featured: 38
    Number of featured: 54
    Number of featured: 72
    Number of featured: 93
    Number of fe

In [23]:
df['shap_median'] = df['shap_iforest_stab_unif_median'].apply(lambda x: np.median(x))

In [24]:
df

,n_estimators,contamination,n_feats,n_iter,n_iter_fs,roc_auc,iforest_stab_unif_median,shap_iforest_stab_unif_median,f1_median,recall_median,precision_median,confusion_matrix,shap_median
0,25,0.094,6,1,1,0.607101,0.895659,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",0.256881,0.212121,0.325581,1,1.000000
1,25,0.094,15,1,1,0.760893,0.931791,"[0.9999999999999919, 0.9999999999999896, 0.999...",0.330275,0.272727,0.418605,1,1.000000
2,25,0.094,25,1,1,0.771707,0.939769,"[0.9999999997906655, 0.9999999997521812, 0.999...",0.366972,0.303030,0.465116,1,1.000000
3,25,0.094,38,1,1,0.802206,0.893935,"[0.999999479792474, 0.9999992058737366, 0.9999...",0.440367,0.363636,0.558140,1,1.000000
4,25,0.094,54,1,1,0.803855,0.875566,"[0.9997510971259926, 0.9995595260076954, 0.999...",0.422018,0.348485,0.534884,1,0.999767
5,25,0.094,72,1,1,0.802127,0.874775,"[0.9682593126226562, 0.9524343005133658, 0.977...",0.403670,0.333333,0.511628,1,0.968340
6,25,0.094,93,1,1,0.805307,0.862139,"[0.47344821504135326, 0.4890318524215279, 0.62...",0.440367,0.363636,0.558140,1,0.584842
7,25,0.094,119,1,1,0.773159,0.899437,"[0.273762063281085, 0.26231833223007905, 0.278...",0.330275,0.272727,0.418605,1,0.297105
8,25,0.094,153,1,1,0.807152,0.875919,"[0.15460133246965146, 0.1292171185462474, 0.13...",0.477064,0.393939,0.604651,1,0.146411
9,25,0.094,233,1,1,0.775632,0.861734,"[0.17438006302698372, 0.21008002769885503, 0.1...",0.422018,0.348485,0.534884,1,0.166679


In [ ]:
results_shap_group.to_parquet(path_shap)

In [ ]:
df = pd.read_parquet(path_shap)
df = df.reset_index()

In [ ]:
df[df.n_feats==254]

# DIFFI

In [ ]:
fs_diffi, fi_diffi, avg_f1_diffi = fs_iforest_with_diffi(train_data, contamination_percentage=contamination_percentage, excluded_cols=excluded_cols, n_iter_fs=n_iter_fs, n_iter=n_iter)

In [ ]:
fi_diffi_all = process_fi(fi_diffi, group)

In [ ]:
fi_diffi_all.to_parquet(path_fi_diffi)

In [ ]:
results_diffi = train_and_evaluate_iforest(train_data, dataset_id, fi_diffi_all, n_tree_estimators=trees, contamination_percentage=contamination_percentage, excluded_cols=excluded_cols, n_iter_fs=n_iter_fs, n_iter=n_iter)

In [ ]:
results_diffi_group = results_diffi.groupby(['n_feats', 'n_estimators', 'contamination']).agg(aggregation_rules)

In [ ]:
results_diffi_group

In [ ]:
results_diffi_group.to_parquet(path_diffi)

### Graph Analysis

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def add_percentage_annotations(ax, x_values, y_values, percentage_values):
    for i, txt in enumerate(percentage_values):
        ax.annotate(f'{round(txt, 3)}%', (x_values.iloc[i], y_values.iloc[i]),
                    textcoords="offset points", xytext=(0, 10), ha='center', va='bottom', color='black')

def plot_feature_importance(df, constant_line1=None, constant_line2=None):
    # Create a figure and a grid of subplots (2 rows, 1 column)
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(30, 20), sharex=True)

    # Top Subplot
    # Plotting F1 Median, Precision, Recall on the primary Y-axis
    ax1.plot(df['cum_value_percentage'], df['f1_median'], marker='o', linestyle='--', color='black', label='F1 Median')
    ax1.plot(df['cum_value_percentage'], df['precision_median'], marker='^', linestyle='-.', color='g', label='Precision')
    ax1.plot(df['cum_value_percentage'], df['recall_median'], marker='v', linestyle=':', color='orange', label='Recall')
    
    # Add the first constant line
    if constant_line1 is not None:
        ax1.axhline(y=constant_line1, color='red', linestyle='-', label='without_fs1')
    
    ax1.set_ylabel('Scores', color='black')
    ax1.tick_params('y', colors='black')
    ax1.legend(loc='upper left')

    # Add Stability to the secondary Y-axis
    ax1_2 = ax1.twinx()
    ax1_2.plot(df['cum_value_percentage'], df['n_feats_percentage'], marker='D', linestyle='-', color='purple', label='% Features')
    ax1_2.set_ylabel('% Features', color='black')
    ax1_2.tick_params('y', colors='black')
    add_percentage_annotations(ax1_2, df['cum_value_percentage'], df['n_feats_percentage'], df['n_feats_percentage'])
    ax1_2.legend(loc='upper right')

    # Bottom Subplot
    # Bar plot on the primary Y-axis
    bars = ax2.bar(df['cum_value_percentage'], df['n_feats'], label='# Features', color='purple')
    # add value annotations on top of each bar
    for bar, value in zip(bars, df['n_feats']):
        ax2.annotate(str(value), xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
                     ha='center', va='bottom', color='black')
    
    # Add the second constant line to the second Y-axis
    ax2_2 = ax2.twinx()
    if constant_line2 is not None:
        ax2_2.axhline(y=constant_line2, color='blue', linestyle='-', label='without_fs2')

    ax2.set_xlabel('Feature importance by SHAP')
    ax2.set_ylabel('# Features')

    # Add Stability to the secondary Y-axis
    ax2_2.plot(df['cum_value_percentage'], df['model_stab_median'], color='b', label='% Stability model', marker='o', linestyle='--')
    ax2_2.plot(df['cum_value_percentage'], df['shap_stab_median'], color='b', label='% Stability model', marker='o', linestyle='--')
    ax2_2.set_ylabel('% Stability model', color='b')
    ax2_2.tick_params('y', colors='b')
    add_percentage_annotations(ax2_2, df['cum_value_percentage'], df['model_stab_median'], df['model_stab_median'])
    ax2.legend(loc='upper right')
    ax2_2.legend(loc='upper left')

    # Set X-axis ticks to the exact values from the 'cum_value_percentage' column for both subplots
    plt.xticks(round(df['cum_value_percentage'], 0))

    # Display the plot
    plt.show()

In [ ]:
data_shap = pd.read_parquet(path_shap)
data_shap = data_shap.reset_index()
fi_shap_all = pd.read_parquet(path_fi_shap)

In [ ]:
# Sample DataFrame
df = pd.merge(data_shap, fi_shap_all, on='n_feats', how='inner')

In [ ]:
df_graph = df[(df.n_estimators==100) & (df.contamination==0.182)]

In [ ]:
# Call the function with the constant lines parameters
plot_feature_importance(df_graph)

In [ ]:
df.head()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_variables(df, variable_configs):
    # Grouping by 'n_estimators' and 'contamination'
    grouped = df.groupby(['n_estimators', 'contamination'])

    # Set up the subplot grid
    fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(15, 15), sharex=True, sharey=True)

    # Flatten the 2D array of subplots for easy indexing
    axes = axes.flatten()

    # Initialize variables to store min and max values for setting axis limits
    min_n_feats, max_n_feats = float('inf'), float('-inf')
    min_variable, max_variable = float('inf'), float('-inf')

    for i, (group, data) in enumerate(grouped):
        n_estimators, contamination = group

        # Plotting on the i-th subplot for each variable in the list
        for variable_name, y_axis in variable_configs:
            color = plt.cm.viridis(variable_configs.index((variable_name, y_axis)) / len(variable_configs))  # Use a colormap for line colors
            if y_axis == 1:
                axes[i].plot(data['n_feats'], data[variable_name], label=f'{variable_name}', marker='o', color=color)
            elif y_axis == 2:
                axes[i].twinx().plot(data['n_feats'], data[variable_name], label=f'{variable_name}', marker='o', color=color)

            # Add a vertical line at the maximum value of each variable with the same color
            max_variable_value = data[variable_name].max()
            max_variable_n_feats = data.loc[data[variable_name].idxmax(), 'n_feats']
            axes[i].axvline(x=max_variable_n_feats, color=color, linestyle='--', label=f'Max {variable_name} at {max_variable_n_feats:.2f}')

            # Update min and max values for setting axis limits
            min_n_feats = min(min_n_feats, data['n_feats'].min())
            max_n_feats = max(max_n_feats, data['n_feats'].max())
            min_variable = min(min_variable, data[variable_name].min())
            max_variable = max(max_variable, data[variable_name].max())

        # Set background color for the subplot with the highest value in the first variable
        max_variable_name, max_y_axis = variable_configs[0]
        max_group = df.loc[df[max_variable_name].idxmax(), ['n_estimators', 'contamination']]
        if group[0] == max_group['n_estimators'] and group[1] == max_group['contamination']:
            axes[i].set_facecolor('lightblue')

        # Customize the subplot
        axes[i].set_title(f'n_estimators={n_estimators}, contamination={contamination}')
        axes[i].set_xlabel('n_feats')
        axes[i].grid(True)

    # Set the same scale for the X-axis and Y-axis in all subplots
    for ax in axes:
        ax.set_xlim(min_n_feats * 0.8, max_n_feats * 1.1)
        ax.set_ylim(min_variable * 0.8, max_variable * 1.2)

        # Set labels for X-axis and Y-axis
        ax.set_xlabel('n_feats')
        ax.set_ylabel('Score')

    # Add a common legend
    lines, labels = axes[0].get_legend_handles_labels()
    fig.legend(lines, labels, loc='upper right', bbox_to_anchor=(0.95, 0.95))

    # Adjust layout for better spacing
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])

    # Show the plot
    plt.show()

In [ ]:
# Assuming your DataFrame is named df
# You can replace df with your actual DataFrame variable
# Specify the list of variables you want to plot (e.g., ['f1_median', 'roc_auc', 'another_variable'])
plot_variables(df, [('f1_median', 1), ('shap_stab_median', 2), ('model_stab_median', 2), ('shap_std_median', 2)])

In [ ]:
data_diffi = pd.read_parquet(path_diffi)
data_diffi = data_diffi.reset_index()
fi_diffi_all= pd.read_parquet(path_fi_diffi)

In [ ]:
# Sample DataFrame
df = pd.merge(data_diffi, fi_diffi_all, on='n_feats', how='inner')
# df = df[df.contamination == 0.161]

# Call the function with the constant lines parameters
plot_feature_importance(df, constant_line1=f1_without_fs, constant_line2=stability_without_fs)

In [ ]:
import pandas as pd
import numpy as np

# Assuming 'df_cor' is your DataFrame
df_cor = data.copy()
df_cor = df_cor.loc[:, ~df_cor.columns.isin(excluded_cols)]

# Replace this with your actual DataFrame
correlation_matrix = df_cor.corr(method='spearman')

# Create a mask to hide the diagonal and the upper triangle of the correlation matrix (since it's symmetric)
mask = (correlation_matrix
        .mask(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
        .abs()
        .stack()
        .sort_values(ascending=False)
        .reset_index()
        .rename(columns={0: 'Correlation'})
        .query('level_0 != level_1'))  # Exclude pairs where the feature is correlated with itself

# Display the top N most correlated pairs (excluding diagonal and self-correlation)
N = 10  # Set the number of top pairs you want to display
mask.head(20)

In [ ]:
fi_diffi_df = fi_diffi.copy()
fi_shap_df = fi_shap.copy()

fi_diffi_df['rank_diffi'] = fi_diffi_df['value'].rank()
fi_shap_df['rank_shap'] = fi_shap_df['value'].rank()

df = pd.merge(fi_shap_df[['feature', 'rank_shap']], fi_diffi_df[['feature', 'rank_diffi']], on='feature', how='inner')


import pandas as pd
import matplotlib.pyplot as plt

# Example DataFrame

# Scatter plot
plt.figure(figsize=(10, 8))
plt.scatter(df['rank_shap'], df['rank_diffi'], marker='o', color='blue')

# Add labels and title
plt.xlabel('Rank in SHAP')
plt.ylabel('Rank in DIFFI')
plt.title('Comparison of Feature Order between DataFrames')

# Annotate points with feature names
for i, feature in enumerate(df['feature']):
    plt.annotate(feature, (df['rank_shap'].iloc[i], df['rank_diffi'].iloc[i]), textcoords="offset points", xytext=(5, 5), ha='left')

# Show the plot
plt.show()

### Test best model with fs by SHAP

In [ ]:
df = pd.merge(data_shap, fi_shap_all, on='n_feats', how='inner')
contamination = df.contamination.unique()[0]
selected_var = list(df.sort_values('f1_median', ascending=False)[:1].feat_selected)[0].tolist()

In [ ]:
df.sort_values('f1_median', ascending=False)[:1]

In [ ]:
hyper = fs_datasets_hyperparams(dataset_id)
hyper['contamination'] = contamination

df = train_data[selected_var]

X = np.array(df)
y = np.array(train_data['y'])
feature_names = np.array(df.columns)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

X_train = pd.DataFrame(X_train, columns=feature_names)
X_test = pd.DataFrame(X_test, columns=feature_names)

model, y_pred, y_scores, y_decision = train_and_predict_isolation_forest(X_train, hyper, excluded_cols=None, random_state=12345)

model.fit(X_train)

### SHAP values 

In [ ]:
explainer = shap.TreeExplainer(model)

In [ ]:
shap_values = explainer(X_test)

df_shap_values = pd.DataFrame(
    shap_values.values, columns=feature_names + ['_shap']
)

df_data = pd.DataFrame(
    shap_values.data, columns=feature_names
)

df = pd.merge(df_data, df_shap_values, left_index=True, right_index=True)

df_cor = df.copy()

df['y'] = y_test
df['y_pred'] = model.predict(X_test)
df['y_pred'] = df.apply(def_outlier, axis=1)
df['y_score'] = -model.score_samples(X_test)
df['score'] = -model.decision_function(X_test)
df["sum_shap"] = df[df[feature_names + '_shap'].columns].sum(axis=1)
df["base_value"] = df["score"] - df["sum_shap"]

anomaly_shap_values = explainer(df[df.y==1][feature_names])

### FI global SHAP 

In [ ]:
# import shap
shap.initjs()

### Global importance feat

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
shap.plots.beeswarm(anomaly_shap_values)

In [ ]:
shap.plots.force(shap_values)

### Global neg importance feat

In [ ]:
anomaly_fi_global = pd.DataFrame(
    shap_values.values, columns=feature_names
)

fi_global = anomaly_fi_global.abs().mean(axis=0)
fi_global = fi_global.sort_values(0, ascending=False)
fi_global

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_combined_graphs(df, selected_cols, target):
    num_cols = len(selected_cols)

    # Set the style of seaborn
    sns.set(style="whitegrid")

    # Create a figure with multiple subplots
    fig, axes = plt.subplots(2, num_cols, figsize=(5 * num_cols, 8))

    target_col = 'y' if target == 'real' else 'y_pred'

    # Common color constants
    green_color = 'green'
    red_color = 'red'
    blue_color = 'blue'

    # Common scatter plot and annotation code
    def scatter_and_annotate(data, color, axis):
        sns.scatterplot(data=data, x=col, y=f'{col}_shap', color=color, ax=axis, alpha=0.5)
        for idx, row in data.iterrows():
            axis.text(row[col], row[f'{col}_shap'], row[col], ha='left', size='small', color=color)

    for i, col in enumerate(selected_cols):
        # Plot 2D Density Plot (Heatmap)
        sns.kdeplot(data=df, x=col, y=f'{col}_shap', cmap='Blues', fill=True, ax=axes[0, i])
        axes[0, i].set_title(f'2D Density Plot (Heatmap) - {col}')

        # Scatter plots and annotations for y=0 and y=1
        scatter_and_annotate(df[df[target_col] == 0], green_color, axes[0, i])
        scatter_and_annotate(df[df[target_col] == 1], red_color, axes[0, i])

        # Store X-axis limits for the first row
        x_limits = axes[0, i].get_xlim()

        # Density Plot
        sns.kdeplot(data=df, x=col, fill=False, color=blue_color, alpha=0.5, ax=axes[1, i])
        sns.kdeplot(data=df[df[target_col] == 1], x=col, fill=False, color=red_color, alpha=0.5, ax=axes[1, i])
        sns.kdeplot(data=df[df[target_col] == 0], x=col, fill=False, color=green_color, alpha=0.5, ax=axes[1, i])
        axes[1, i].set_title(f'Density Plot of {col}')
        axes[1, i].set_xlabel(col)
        axes[1, i].set_ylabel('Density')

        # Set X-axis limits for the second row based on the first row
        axes[1, i].set_xlim(x_limits)

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Show the plot
    plt.show()


In [ ]:
n_cols = 3
selected_cols = list(fi_global[0:n_cols].index)
selected_cols

In [ ]:
# Example usage with multiple columns
plot_combined_graphs(df, selected_cols, 'real')

In [ ]:
plot_combined_graphs(df, selected_cols, 'pred')

In [ ]:
anomaly_df = df[(df.y==1)][list(feature_names) + ['y', 'y_pred', 'score', 'y_score', 'base_value', 'sum_shap']]
anomaly_df[(anomaly_df.Col95==84.0)]

In [ ]:
id_pr = 61
var = ['Col7', 'Col107', 'Col119', 'Col95']

In [ ]:
df[df.index==id_pr][var]

In [ ]:
def local_pr_shap(id_pr, df, feature_columns):
    fs = feature_columns.tolist()
    fs_shap = feature_columns + '_shap'
    fs_shap = fs_shap.tolist()

    df_local = df[feature_columns][df.index == id_pr]
    base_value = df["base_value"][df.index == id_pr]

    shap_values = df[fs_shap][df.index == id_pr]
    shap_values = shap_values.values.ravel()

    return df_local, shap_values, base_value.iloc[0]


def local_shap_table(id_pred, df_local, feature_columns):
    
    fs = feature_columns.tolist()
    fs_shap = feature_columns + '_shap'
    fs_shap = fs_shap.tolist()

    df_local = df[feature_columns][df.index == id_pr]
    base_value = df["base_value"][df.index == id_pr]

    shap_values_local = df[fs_shap][df.index == id_pr]

    shap_temp = pd.DataFrame(shap_values_local, columns=fs_shap)
    shap_temp.columns = shap_temp.columns.str.replace('_shap', '')
    values = pd.DataFrame(df_local, columns=fs)

    shap_temp = pd.melt(shap_temp, value_vars=df_local.columns)
    shap_temp = shap_temp.rename(columns={"value": "shap_value"})
    values = pd.melt(values, value_vars=df_local.columns)

    table = pd.merge(
        values, shap_temp, how="left", left_on="variable", right_on="variable"
    )

    return table.sort_values("shap_value", ascending=True)

In [ ]:
df_local, shap_values_local, base_value = local_pr_shap(id_pr, df, feature_names)

In [ ]:
df_local.head()

In [ ]:
shap.plots.force(-base_value, -shap_values_local, df_local, matplotlib = True)

In [ ]:
shap.plots.waterfall(anomaly_shap_values[10], max_display=12)

In [ ]:
table = local_shap_table(id_pr, df_local, feature_names)
table.head(n_cols)

In [ ]:
n_cols = 4
selected_cols = list(table[0:n_cols].variable)
selected_cols

In [ ]:
plot_combined_graphs(df, selected_cols, 'real')
plot_combined_graphs(df, selected_cols, 'pred')

In [ ]:
# Example usage
factors_to_test = [0.97, 1, 1.03] # Add more factors if needed

detail_table, table_out = generate_shap_tables(df, feature_names, explainer, factors_to_test, beta_flavor=1, gamma=1.4, psi=0.8)

In [ ]:
import numpy as np

# Assuming your DataFrame is named result_tables
# You can replace result_tables with your actual DataFrame variable

nte = len(factors_to_test)
random_stdev = np.sqrt((nte + 1) * (nte - 1) / (12 * nte**2))

# Calculate stability_scores_id for every different id
result_tables['stability_scores_id'] = result_tables.groupby('id').apply(lambda group: np.mean(np.minimum(1, group['point_stabilities'] / random_stdev))).reset_index(level=0, drop=True)

# Select the required columns
result = result_tables[['id', 'stability_scores_id']].drop_duplicates().reset_index(drop=True)
result = result.dropna()
result['stability_scores_id'] = 1 - result['stability_scores_id']
# Display the result
result.describe()


## Credit card

**Dataset source**: https://www.kaggle.com/mlg-ulb/creditcardfraud

**Additional sources:**

Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

Dal Pozzolo, Andrea; Caelen, Olivier; Le Borgne, Yann-Ael; Waterschoot, Serge; Bontempi, Gianluca. Learned lessons in credit card fraud detection from a practitioner perspective, Expert systems with applications,41,10,4915-4928,2014, Pergamon

Dal Pozzolo, Andrea; Boracchi, Giacomo; Caelen, Olivier; Alippi, Cesare; Bontempi, Gianluca. Credit card fraud detection: a realistic modeling and a novel learning strategy, IEEE transactions on neural networks and learning systems,29,8,3784-3797,2018,IEEE

Dal Pozzolo, Andrea Adaptive Machine learning for credit card fraud detection ULB MLG PhD thesis (supervised by G. Bontempi)

Carcillo, Fabrizio; Dal Pozzolo, Andrea; Le Borgne, Yann-Aël; Caelen, Olivier; Mazzer, Yannis; Bontempi, Gianluca. Scarff: a scalable framework for streaming credit card fraud detection with Spark, Information fusion,41, 182-194,2018,Elsevier

Carcillo, Fabrizio; Le Borgne, Yann-Aël; Caelen, Olivier; Bontempi, Gianluca. Streaming active learning strategies for real-life credit card fraud detection: assessment and visualization, International Journal of Data Science and Analytics, 5,4,285-300,2018,Springer International Publishing

Bertrand Lebichot, Yann-Aël Le Borgne, Liyun He, Frederic Oblé, Gianluca Bontempi Deep-Learning Domain Adaptation Techniques for Credit Cards Fraud Detection, INNSBDDL 2019: Recent Advances in Big Data and Deep Learning, pp 78-88, 2019

Fabrizio Carcillo, Yann-Aël Le Borgne, Olivier Caelen, Frederic Oblé, Gianluca Bontempi Combining Unsupervised and Supervised Learning in Credit Card Fraud Detection Information Sciences, 2019

Yann-Aël Le Borgne, Gianluca Bontempi Machine Learning for Credit Card Fraud Detection - Practical Handbook

In [ ]:
data = pd.read_csv('./creditcard.csv')

In [ ]:
data = data.drop(columns = ['Time'])

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
pd.pivot_table(data,
             values = 'V1',
               index = 'Class', 
              aggfunc = 'count')

### iForest

In [ ]:
train_data = data.copy()

In [ ]:
start = time.process_time()

clf = IsolationForest(max_samples=256, n_estimators=100)
clf.fit(train_data.loc[:, train_data.columns != 'Class'])

end = time.process_time()
creditcard_iforest_train_time = end - start
print(end - start)

start = time.process_time()

y_pred = clf.predict(train_data.loc[:, train_data.columns != 'Class'])
y_scores = clf.score_samples(train_data.loc[:, train_data.columns != 'Class'])

end = time.process_time()
creditcard_iforest_test_time = end - start
print(end - start)

In [ ]:
train_data['y_pred'] = y_pred
train_data['prediction'] = train_data.apply(def_outlier, axis = 1)
train_data['y_scores'] = -y_scores

In [ ]:
confusion_matrix(train_data['Class'], train_data['prediction'])

In [ ]:
fpr, tpr, _ = metrics.roc_curve(train_data['Class'], train_data['y_scores'])
creditcard_iforest_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

In [ ]:
creditcard_iforest_report = classification_report(train_data['Class'], train_data['prediction'], target_names = ['0','1'], output_dict = True)
print(classification_report(train_data['Class'], train_data['prediction'], target_names = ['0','1']))

In [ ]:
print(creditcard_iforest_report['1']['precision'])
print(creditcard_iforest_report['1']['recall'])
print(creditcard_iforest_report['1']['f1-score'])

In [ ]:
precision, recall, thresholds = precision_recall_curve(train_data['Class'], train_data['y_scores'])
creditcard_iforest_auc_precision_recall = metrics.auc(recall, precision)
print(creditcard_iforest_auc_precision_recall)

## bank

**Dataset source**: https://github.com/GuansongPang/ADRepository-Anomaly-detection-datasets/tree/main/categorical%20data

Pang, G., Shen, C., Cao, L., & Hengel, A. V. D. (2021). Deep learning for anomaly detection: A review. ACM Computing Surveys (CSUR), 54(2), 1-38.

In [ ]:
data = pd.read_csv('./bank.csv')

In [ ]:
data.head()

In [ ]:
pd.pivot_table(data,
             values = 'age',
               index = 'class', 
              aggfunc = 'count')

### iForest

In [ ]:
train_data = data.copy()

In [ ]:
start = time.process_time()

clf = IsolationForest(max_samples = 256, n_estimators = 100)
clf.fit(train_data.loc[:, train_data.columns != 'class'])

end = time.process_time()
bank_iforest_train_time = end - start
print(end - start)

start = time.process_time()

y_pred = clf.predict(train_data.loc[:, train_data.columns != 'class'])
y_scores = clf.score_samples(train_data.loc[:, train_data.columns != 'class'])
end = time.process_time()
bank_iforest_test_time = end - start
print(end - start)

In [ ]:
train_data['y_pred'] = y_pred
train_data['prediction'] = train_data.apply(def_outlier, axis = 1)
train_data['y_scores'] = -y_scores

In [ ]:
confusion_matrix(train_data['class'], train_data['prediction'])

In [ ]:
fpr, tpr, _ = metrics.roc_curve(train_data['class'], train_data['y_scores'])
bank_iforest_auc = metrics.auc(fpr, tpr)
metrics.auc(fpr, tpr)

In [ ]:
bank_iforest_report = classification_report(train_data['class'], train_data['prediction'], target_names = ['0','1'], output_dict=True)
print(classification_report(train_data['class'], train_data['prediction'], target_names = ['0','1']))

In [ ]:
print(bank_iforest_report['1']['precision'])
print(bank_iforest_report['1']['recall'])
print(bank_iforest_report['1']['f1-score'])

In [ ]:
precision, recall, thresholds = precision_recall_curve(train_data['class'], train_data['y_scores'])
bank_iforest_auc_precision_recall = metrics.auc(recall, precision)
print(bank_iforest_auc_precision_recall)

## Performance

In [ ]:
performance = pd.DataFrame(columns = ['F1 score', 'recall', 'precision', 'AUC', 'AUPRC', 
                                      'Training time','Inference time','Total time'])

In [ ]:
f1_score_iforest = {'arrhythmia':arrhythmia_iforest_report['1']['f1-score'],
                       'cardio':cardio_iforest_report['1']['f1-score'], 
                        'forestcover':forestcover_iforest_report['1']['f1-score'], 
                       'annthyroid':annthyroid_iforest_report['1']['f1-score'],       
                        'creditcard':creditcard_iforest_report['1']['f1-score'], 
                       'mammography':mammography_iforest_report['1']['f1-score'], 
                        'shuttle':shuttle_iforest_report['1']['f1-score'], 
                      'mnist':mnist_iforest_report['1']['f1-score'], 
                  'vowels':vowels_iforest_report['1']['f1-score'], 
                  'seismic':seismic_iforest_report['1']['f1-score'], 
                  'musk':musk_iforest_report['1']['f1-score'], 
                  'bank':bank_iforest_report['1']['f1-score']}
f1_score_iforest_df = pd.DataFrame.from_dict(f1_score_iforest, orient='index', columns = ['F1 score']).reset_index()

In [ ]:
recall_iforest = {'arrhythmia':arrhythmia_iforest_report['1']['recall'],
                       'cardio':cardio_iforest_report['1']['recall'], 
                        'forestcover':forestcover_iforest_report['1']['recall'], 
                       'annthyroid':annthyroid_iforest_report['1']['recall'],       
                        'creditcard':creditcard_iforest_report['1']['recall'], 
                       'mammography':mammography_iforest_report['1']['recall'], 
                        'shuttle':shuttle_iforest_report['1']['recall'], 
                      'mnist':mnist_iforest_report['1']['recall'], 
                  'vowels':vowels_iforest_report['1']['recall'], 
                  'seismic':seismic_iforest_report['1']['recall'], 
                  'musk':musk_iforest_report['1']['recall'], 
                  'bank':bank_iforest_report['1']['recall'], }
recall_iforest_df = pd.DataFrame.from_dict(recall_iforest, orient='index', columns = ['Recall']).reset_index()

In [ ]:
precision_iforest = {'arrhythmia':arrhythmia_iforest_report['1']['precision'],
                       'cardio':cardio_iforest_report['1']['precision'], 
                        'forestcover':forestcover_iforest_report['1']['precision'], 
                       'annthyroid':annthyroid_iforest_report['1']['precision'],       
                        'creditcard':creditcard_iforest_report['1']['precision'], 
                       'mammography':mammography_iforest_report['1']['precision'], 
                        'shuttle':shuttle_iforest_report['1']['precision'], 
                      'mnist':mnist_iforest_report['1']['precision'], 
                  'vowels':vowels_iforest_report['1']['precision'], 
                  'seismic':seismic_iforest_report['1']['precision'], 
                  'musk':musk_iforest_report['1']['precision'], 
                  'bank':bank_iforest_report['1']['precision'], }
precision_iforest_df = pd.DataFrame.from_dict(precision_iforest, orient='index', columns = ['Precision']).reset_index()

In [ ]:
auc_iforest = {'arrhythmia':arrhythmia_iforest_auc,
                       'cardio':cardio_iforest_auc, 
                        'forestcover':forestcover_iforest_auc, 
                       'annthyroid':annthyroid_iforest_auc,       
                        'creditcard':creditcard_iforest_auc, 
                       'mammography':mammography_iforest_auc, 
                        'shuttle':shuttle_iforest_auc, 
                      'mnist':mnist_iforest_auc, 
                  'vowels':vowels_iforest_auc, 
                  'seismic':seismic_iforest_auc, 
                  'musk':musk_iforest_auc, 
                  'bank':bank_iforest_auc}
auc_iforest_df = pd.DataFrame.from_dict(auc_iforest, orient='index', columns = ['AUC']).reset_index()

In [ ]:
auprc_iforest = {'arrhythmia':arrhythmia_iforest_auc_precision_recall,
                       'cardio':cardio_iforest_auc_precision_recall, 
                        'forestcover':forestcover_iforest_auc_precision_recall, 
                       'annthyroid':annthyroid_iforest_auc_precision_recall,       
                        'creditcard':creditcard_iforest_auc_precision_recall, 
                       'mammography':mammography_iforest_auc_precision_recall, 
                        'shuttle':shuttle_iforest_auc_precision_recall, 
                      'mnist':mnist_iforest_auc_precision_recall, 
                  'vowels':vowels_iforest_auc_precision_recall, 
                  'seismic':seismic_iforest_auc_precision_recall, 
                  'musk':musk_iforest_auc_precision_recall, 
                  'bank':bank_iforest_auc_precision_recall}
auprc_iforest_df = pd.DataFrame.from_dict(auprc_iforest, orient='index', columns = ['AUPRC']).reset_index()

In [ ]:
training_time_iforest = {'arrhythmia':arrhythmia_iforest_train_time,
                       'cardio':cardio_iforest_train_time, 
                        'forestcover':forestcover_iforest_train_time, 
                       'annthyroid':annthyroid_iforest_train_time,       
                        'creditcard': creditcard_iforest_train_time, 
                       'mammography':mammography_iforest_train_time, 
                        'shuttle':shuttle_iforest_train_time, 
                      'mnist':mnist_iforest_train_time, 
                  'vowels':vowels_iforest_train_time, 
                  'seismic':seismic_iforest_train_time, 
                  'musk':musk_iforest_train_time, 
                  'bank':bank_iforest_train_time}
training_time_iforest_df = pd.DataFrame.from_dict(training_time_iforest, orient='index', columns = ['Training time']).reset_index()

In [ ]:
test_time_iforest = {'arrhythmia':arrhythmia_iforest_test_time,
                       'cardio':cardio_iforest_test_time, 
                        'forestcover':forestcover_iforest_test_time, 
                       'annthyroid':annthyroid_iforest_test_time,       
                        'creditcard':creditcard_iforest_test_time, 
                       'mammography':mammography_iforest_test_time, 
                        'shuttle':shuttle_iforest_test_time, 
                      'mnist':mnist_iforest_test_time, 
                  'vowels':vowels_iforest_test_time, 
                  'seismic':seismic_iforest_test_time, 
                  'musk':musk_iforest_test_time, 
                  'bank':bank_iforest_test_time}
test_time_iforest_df = pd.DataFrame.from_dict(test_time_iforest, orient='index', columns = ['Testing time']).reset_index()

In [ ]:
total_time_iforest = {'arrhythmia':arrhythmia_iforest_train_time + arrhythmia_iforest_test_time,
                       'cardio':cardio_iforest_train_time + cardio_iforest_test_time, 
                        'forestcover':forestcover_iforest_train_time + forestcover_iforest_test_time, 
                       'annthyroid':annthyroid_iforest_train_time + annthyroid_iforest_test_time,       
                        'creditcard': creditcard_iforest_train_time + creditcard_iforest_test_time, 
                       'mammography':mammography_iforest_train_time + mammography_iforest_test_time, 
                        'shuttle':shuttle_iforest_train_time + shuttle_iforest_test_time, 
                      'mnist':mnist_iforest_train_time + mnist_iforest_test_time, 
                  'vowels':vowels_iforest_train_time + vowels_iforest_test_time, 
                  'seismic':seismic_iforest_train_time + seismic_iforest_test_time, 
                  'musk':musk_iforest_train_time + musk_iforest_test_time, 
                  'bank':bank_iforest_train_time + bank_iforest_test_time}
total_time_iforest_df = pd.DataFrame.from_dict(total_time_iforest, orient='index', columns = ['Total time']).reset_index()

In [ ]:
pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(f1_score_iforest_df, recall_iforest_df, how = 'inner'), 
                                    precision_iforest_df, how ='inner'),
         auc_iforest_df, how = 'inner'), auprc_iforest_df, how = 'inner'), training_time_iforest_df, how = 'inner'), 
         test_time_iforest_df, how = 'inner'),total_time_iforest_df, how = 'inner')